In [1]:
import os
import ast
from dotenv import load_dotenv
import arxiv

from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [2]:
from openai import OpenAI
client = OpenAI()

In [ ]:
chat_model = ChatOpenAI( model_kwargs={"stop": ["Observation:"]},)

## Conversation Memories

#### A- ConversationBufferMemory

In [3]:
prompt = """System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation:
{history}
Human: {input}
AI:"""

history = ''

def talk(question):

    global history
    
    prompt_after_formatting = prompt.format(input=question,
                                            history=history)

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt_after_formatting}
        ]
    )

    output = completion.choices[0].message.content

    history += f"Human: {question}\nAI: {output}\n"

    return output
    

In [4]:
talk("How are you today?")
talk("Sorry. What did I ask you?")
talk("Do you know anything about football?")
print(prompt.format(input="current question",history=history))

System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation:
Human: How are you today?
AI: I'm doing well, thank you for asking! I've been learning more about machine learning algorithms today. Did you know that neural networks are inspired by the way the human brain processes information? It's fascinating how technology can mimic biological processes.
Human: Sorry. What did I ask you?
AI: You asked me how I was doing today. And I shared with you that I've been learning more about machine learning algorithms, specifically neural networks inspired by the human brain. It's amazing how technology can replicate biological processes, don't you think?
Human: Do you know anything about football?
AI: Yes, I know about football! It's a popular sport played around the world. The rules of the game involve two teams trying to score goals by getting the ball into the opposing team's goal using any

### B- ConversationSummaryMemory

In [5]:
prompt = """System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
{summary}
Human: {input}
AI:"""

summary_prompt = """System: Your task is to summarize below conversation with emphasis on key points. If nothing given, return ''.
{history}
Summary:"""

history = ''

def talk(question):

    global history

    summary_prompt_after_formatting = summary_prompt.format(history=history)

    summary_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": summary_prompt_after_formatting}
        ]
    )
    
    summary = summary_completion.choices[0].message.content

    prompt_after_formatting = prompt.format(input=question,
                                            summary=summary)
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt_after_formatting}
        ]
    )

    output = completion.choices[0].message.content
    print(prompt_after_formatting)
    history += f"Human: {question}\nAI: {output}\n"

    return output

In [6]:
talk("How are you today?")

System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
"Nothing."
Human: How are you today?
AI:


"Hello! I'm doing well, thank you for asking. How can I assist you today?"

In [7]:
talk("Sorry. What did I ask you?")

System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
AI is doing well and ready to assist the human with any task.
Human: Sorry. What did I ask you?
AI:


'You asked me to explain the concept of object-oriented programming and how it can be applied in software development. Would you like me to go over that again for you?'

In [8]:
talk("do you know about football?")

System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
The AI explained that it is doing well and asked how it can assist the human. The human asked the AI to explain the concept of object-oriented programming and how it can be applied in software development.
Human: do you know about football?
AI:


'Yes, I am aware of football. It is a popular sport played worldwide. Would you like to know more about it or how it relates to our conversation?'

In [9]:
talk("when did england compete in euros first?")

System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
The human asked the AI how it was doing and then inquired about football. The AI offered to assist with explaining object-oriented programming in software development but the human did not pursue that topic further.
Human: when did england compete in euros first?
AI:


'England first competed in the UEFA European Championship (Euros) in 1968.'

In [10]:
talk("what were the other teams in the same competition?")

System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
The conversation covers topics ranging from the AI's readiness to assist, object-oriented programming, football, and England's first participation in the UEFA European Championship in 1968.
Human: what were the other teams in the same competition?
AI:


'In the 1968 UEFA European Championship, the other teams that participated were the Soviet Union, Yugoslavia, and Italy. England was the eventual winners of the tournament, defeating the Soviet Union in the final.'

## Function-Use (Agents and Tools)

In [26]:
def arxiv_search(args):
    # Construct the default API client.
    client = arxiv.Client()

    # Search for the paper with ID
    search_by_id = arxiv.Search(query=args['query'],
                                id_list=[args['id']])
    # Reuse client to fetch the paper, then print its title.
    first_result = next(client.results(search_by_id))
    
    return first_result.title

def convert_time(args):
    temp = args['time'].split(':')

    return int(temp[0])*3600 + int(temp[1])*60 + int(temp[2])

In [27]:
system_prompt = """Answer the following questions as best you can. You have access to the following tools:

{tools}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do. only one action at a time.
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer.

Question: {input}
Thought:{agent_scratchpad}"""

tools = {
    'convert_time': 'A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}',
    'arxiv_search': 'A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}'
}

In [34]:
agent_scratchpad = ''

def ask(question):

    global agent_scratchpad

    tools_str = ''

    for func, desc in tools.items():
        tools_str += f"{func} : {desc}\n"
    
    prompt_after_formatting = system_prompt.format(input=question, 
                                                   agent_scratchpad=agent_scratchpad, 
                                                   tools=tools_str, 
                                                   tool_names=list(tools.keys()))
    print(prompt_after_formatting)
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt_after_formatting}
        ],
        stop=["Observation:"]
    )
    output = completion.choices[0].message.content
    print(output)
    
    while output.find("Final Answer: ")==-1:

        agent_scratchpad += output

        function_name = output.split("Action: ")[1].split('\n')[0]
        function_args = ast.literal_eval(output.split("Action Input: ")[1].split('\n')[0])

        result = globals()[function_name](function_args)
        agent_scratchpad += f"Observation: {result}\n"
        print(f"Observation: {result}\n")
        
        prompt_after_formatting = system_prompt.format(input=question, 
                                                    agent_scratchpad=agent_scratchpad, 
                                                    tools=tools_str, 
                                                    tool_names=list(tools.keys()))
        
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt_after_formatting}
            ],
            stop=["Observation:"]
        )
        output = completion.choices[0].message.content
        
        print(output)  
    
    return output

In [35]:
ask("First answer how many seconds in 4:00:01. Then answer What's the paper 1605.08386 about?")

Answer the following questions as best you can. You have access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
arxiv_search : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}


ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do. only one action at a time.
Action: the action to take, should be one of ['convert_time', 'arxiv_search']
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer.

Question: First answer how many seconds in 4:00:01. Then 

'Final Answer: The time 4:00:01 is equivalent to 14401 seconds. The paper 1605.08386 is about Heat-bath random walks with Markov bases.'